In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit

In [2]:
pd.read_csv("csvs/bigbench_newschema.csv")['model'].unique()

array(['BIG-G_T=1_2b', 'PaLM_535b', 'BIG-G_T=1_2m', 'BIG-G_T=1_422m',
       'BIG-G_sparse_2m', 'GPT_GPT-3_Medium', 'BIG-G_T=1_128b',
       'BIG-G_T=0_16m', 'GPT_GPT-3_200B', 'BIG-G_sparse_8b',
       'BIG-G_T=0_422m', 'GPT_GPT-3_Small', 'BIG-G_sparse_53m',
       'BIG-G_T=0_53m', 'BIG-G_T=0_1b', 'BIG-G_sparse_125m',
       'BIG-G_T=1_4b', 'BIG-G_sparse_4b', 'BIG-G_T=0_8b', 'BIG-G_T=1_53m',
       'BIG-G_sparse_422m', 'BIG-G_T=1_27b', 'PaLM_64b', 'BIG-G_T=1_8b',
       'BIG-G_T=0_27b', 'BIG-G_sparse_16m', 'PaLM_8b', 'BIG-G_T=0_2m',
       'BIG-G_T=0_128b', 'BIG-G_T=0_244m', 'GPT_GPT-3_XL',
       'BIG-G_T=0_125m', 'BIG-G_sparse_244m', 'BIG-G_sparse_2b',
       'BIG-G_T=0_2b', 'BIG-G_T=1_244m', 'GPT_GPT-3_Large',
       'GPT_GPT-3_3B', 'BIG-G_T=1_16m', 'BIG-G_sparse_1b',
       'BIG-G_T=1_125m', 'BIG-G_T=0_4b', 'GPT_GPT-3_6B', 'BIG-G_T=1_1b',
       'GPT_GPT-3_13B', 'Gopher_44M', 'Gopher_280B', 'Gopher_417M',
       'Gopher_117M', 'Gopher_1.4B', 'Gopher_7.1B', 'T0_T0', 'T0++_T0++',
   

In [3]:
# csv = 
df = pd.read_csv('csvs/bigbench_newschema_preferred_mcq_grade.csv')
# drop null rows
df = df.replace([np.inf, -np.inf], np.nan)
df = df.dropna()
# df = df[df['number_of_shots'].isin([0, 1, 2, 3])]
train_df = df.sample(int(0.8 * len(df)))
test_df = df.drop(train_df.index)
numerical_df = df[['number_of_shots', 'temperature', 'non_embedding_params', 'total_params', 'training_batch_size', 'training_steps', 'score']]
print("Train size", len(train_df), "Test size", len(test_df))
print(numerical_df.corr().to_markdown())
df.describe()

Train size 43049 Test size 10763
|                      |   number_of_shots |   temperature |   non_embedding_params |   total_params |   training_batch_size |   training_steps |        score |
|:---------------------|------------------:|--------------:|-----------------------:|---------------:|----------------------:|-----------------:|-------------:|
| number_of_shots      |       1           |  -0.00434305  |           -0.000665014 |   -0.000666343 |           -0.00130321 |      -0.00282396 |  0.0389363   |
| temperature          |      -0.00434305  |   1           |           -0.0221554   |   -0.0217895   |           -0.207801   |       0.244261   | -0.000988636 |
| non_embedding_params |      -0.000665014 |  -0.0221554   |            1           |    0.999999    |            0.513901   |       0.419474   |  0.151035    |
| total_params         |      -0.000666343 |  -0.0217895   |            0.999999    |    1           |            0.512993   |       0.419991   |  0.151097    |
|

,number_of_shots,temperature,non_embedding_params,total_params,training_batch_size,training_steps,score
count,53812.000000,53812.000000,5.381200e+04,5.381200e+04,5.381200e+04,5.381200e+04,53812.000000
mean,1.477997,0.289601,1.665473e+10,1.671633e+10,4.764267e+05,6.933628e+05,0.360767
std,1.123665,0.453581,4.101677e+10,4.105028e+10,6.584038e+05,4.934638e+05,0.190447
min,0.000000,0.000000,2.098048e+06,1.029005e+07,2.621440e+05,7.500000e+04,0.000000
25%,0.000000,0.000000,2.181238e+08,2.345078e+08,2.621440e+05,5.000000e+05,0.222222
50%,1.000000,0.000000,1.300000e+09,1.300000e+09,2.621440e+05,5.210000e+05,0.333333
75%,2.000000,1.000000,8.590103e+09,8.721175e+09,2.621440e+05,6.492000e+05,0.500000
max,3.000000,1.000000,2.000000e+11,2.000000e+11,4.000000e+06,2.571500e+06,1.000000


In [4]:
str_eqn1 = 'constant_1 + (constant_2 / (training_steps ^ number_of_shots))'
str_eqn2 = 'constant_1 + (constant_2 / ((training_steps / constant_3) ^ number_of_shots))'
str_chinchilla = '(constant_1 / ((training_steps * training_batch_size) ^ constant_2)) + (constant_3 / (total_params ^ constant_4)) + constant_5'
str_chinchilla_ablation1 = '(constant_1 / ((training_steps * training_batch_size) ^ constant_2)) + constant_3'
str_chinchilla_ablation2 = '(constant_1 / (total_params ^ constant_2)) + constant_3'
str_chinchilla_ablate_numer = '(1 / ((training_steps * training_batch_size) ^ constant_1)) + (1 / (total_params ^ constant_2)) + constant_3'
str_chinchilla_ablate_denom = '(constant_1 / (training_steps * training_batch_size)) + (constant_2 / (total_params)) + constant_3'
str_modified_chinchilla = '(constant_1 / ((training_steps * training_batch_size) ^ (constant_2 * number_of_shots))) + (constant_3 / (total_params ^ constant_4)) + constant_5'

def eqn1(x, constant_1, constant_2):
    training_steps, number_of_shots = x
    return constant_1 + (constant_2 / (training_steps ** number_of_shots))

def eqn2(x, constant_1, constant_2, constant_3):
    training_steps, number_of_shots = x
    return constant_1 + (constant_2 / ((training_steps / constant_3) ** number_of_shots))

def chinchilla(x, constant_1, constant_2, constant_3, constant_4, constant_5):
    training_steps, number_of_shots, total_params, training_batch_size = x
    return (constant_1 / ((training_steps * training_batch_size) ** constant_2)) + (constant_3 / (total_params ** constant_4)) + constant_5

def chinchilla_ablation1(x, constant_1, constant_2, constant_3):
    training_steps, number_of_shots, total_params, training_batch_size = x
    return (constant_1 / ((training_steps * training_batch_size) ** constant_2)) + constant_3

def chinchilla_ablation2(x, constant_1, constant_2, constant_3):
    training_steps, number_of_shots, total_params, training_batch_size = x
    return (constant_1 / (total_params ** constant_2)) + constant_3

def chinchilla_ablate_numer(x, constant_1, constant_2, constant_3):
    training_steps, number_of_shots, total_params, training_batch_size = x
    return (1 / ((training_steps * training_batch_size) ** constant_1)) + (1 / (total_params ** constant_2)) + constant_3

def chinchilla_ablate_denom(x, constant_1, constant_2, constant_3):
    training_steps, number_of_shots, total_params, training_batch_size = x
    return (constant_1 / (training_steps * training_batch_size)) + (constant_2 / total_params) + constant_3

def modified_chinchilla(x, constant_1, constant_2, constant_3, constant_4, constant_5):
    training_steps, number_of_shots, total_params, training_batch_size = x
    return (constant_1 / ((training_steps * training_batch_size) ** (constant_2 * number_of_shots))) + (constant_3 / (total_params ** constant_4)) + constant_5

x = np.vstack([
    train_df['training_steps'].values,
    train_df['number_of_shots'].values,
])

wide_x = np.vstack([
    train_df['training_steps'].values,
    train_df['number_of_shots'].values,
    train_df['total_params'].values,
    train_df['training_batch_size'].values,
])
bounds = (
    [-np.inf, 0, -np.inf, 0, -np.inf],
    [np.inf, np.inf, np.inf, np.inf, np.inf]
)

y = train_df['score'].values

# This fits without a monotonicity constraint. The solution usually devolves into a degenerate solution with flip-flopping constants.
# instead we're going to add a monotonicity constraint to fit the parameters.
# popt1, pcov1, infodict1, mesg1, ier1 = curve_fit(eqn1, x, y, full_output=True)
# popt2, pcov2, infodict2, mesg2, ier2 = curve_fit(eqn2, x, y, full_output=True)
# popt3, pcov3, infodict3, mesg3, ier3 = curve_fit(chinchilla, wide_x, y, full_output=True, bounds=bounds)
# popt4, pcov4, infodict4, mesg4, ier4 = curve_fit(modified_chinchilla, wide_x, y, full_output=True, bounds=bounds)
# popt5, pcov5, infodict5, mesg5, ier5 = curve_fit(chinchilla_ablation1, wide_x, y, full_output=True, bounds=(bounds[0][2:], bounds[1][2:]))
# popt6, pcov6, infodict6, mesg6, ier6 = curve_fit(chinchilla_ablation2, wide_x, y, full_output=True, bounds=(bounds[0][2:], bounds[1][2:]))
# popt7, pcov7, infodict7, mesg7, ier7 = curve_fit(chinchilla_ablate_numer, wide_x, y, full_output=True, bounds=([0, 0, -np.inf], [np.inf, np.inf, np.inf]))
# popt8, pcov8, infodict8, mesg8, ier8 = curve_fit(chinchilla_ablate_denom, wide_x, y, full_output=True, bounds=([-np.inf, -np.inf, -np.inf], [np.inf, np.inf, np.inf]))

In [5]:
from scipy.optimize import minimize

def monotonic_objective(predictor, mix=0.65):
    def fn1(params, xdata, ydata):
        """
        x[0] is the training steps.
        We penalize the model for not being monotonicically increasing with respect to training steps.
        """
        predicted = predictor(xdata, *params)
        p = np.array(predicted[np.argsort(x[0])])
        diff = p[1:] - p[:-1]
        valid_diff = diff[diff < 0]
        monotonic_increase = 0.0 if not len(valid_diff) else valid_diff.mean()

        error = (np.sum((ydata - predicted)**2)) # RMSE loss
        return ((1 - mix) * error) + (mix * monotonic_increase)
    return fn1

kwargs = dict(
    options=dict(disp=False, maxiter=1000),
    method='BFGS',
)
popt1 = minimize(monotonic_objective(eqn1), x0=np.array([0.363219, -0.015540]), args=(x, y), **kwargs).x
popt2 = minimize(monotonic_objective(eqn2), x0=np.array([0.363219, -0.015540, 12605]), args=(x, y), **kwargs).x
popt3 = minimize(monotonic_objective(chinchilla), x0=np.array([1., 0.99983365, 0.8788157 , 3.13948064, 0.36130033]), args=(wide_x, y), **kwargs).x
popt4 = minimize(monotonic_objective(modified_chinchilla), x0=np.array([-0.02186013,  1.00041618,  0.89328067,  2.88247603,  0.36694853]), args=(wide_x, y), **kwargs).x
popt5 = minimize(monotonic_objective(chinchilla_ablation1), x0=np.array([1.        , 1.        , 0.36130362]), args=(wide_x, y), **kwargs).x
popt6 = minimize(monotonic_objective(chinchilla_ablation2), x0=np.array([0.90374502, 2.6071086 , 0.3613004 ]), args=(wide_x, y), **kwargs).x
popt7 = minimize(monotonic_objective(chinchilla_ablate_numer), x0=np.array([0.99987481, 2.60998853, 0.3613004 ]), args=(wide_x, y), **kwargs).x
popt8 = minimize(monotonic_objective(chinchilla_ablate_denom), x0=np.array([ 4.93647189e-01, -6.20734318e+05,  3.66433551e-01]), args=(wide_x, y), **kwargs).x

In [6]:
from functools import partial
# display equation with constants as math
# fitted_eqn = eqn
# for i, constant in enumerate(popt):
#     fitted_eqn = fitted_eqn.replace('constant_{}'.format(i+1), str(constant))
def fit_eqn(popt, eqn):
    fitted_eqn = eqn
    for i, constant in enumerate(popt):
        fitted_eqn = fitted_eqn.replace('constant_{}'.format(i+1), str(constant))
    return fitted_eqn

str_fitted_eqn1 = fit_eqn(popt1, str_eqn1)
str_fitted_eqn2 = fit_eqn(popt2, str_eqn2)
str_fitted_chinchilla = fit_eqn(popt3, str_chinchilla)
str_fitted_modified_chinchilla = fit_eqn(popt4, str_modified_chinchilla)
str_fitted_chinchilla_ablation1 = fit_eqn(popt5, str_chinchilla_ablation1)
str_fitted_chinchilla_ablation2 = fit_eqn(popt6, str_chinchilla_ablation2)
str_fitted_chinchilla_ablate_numer = fit_eqn(popt7, str_chinchilla_ablate_numer)
str_fitted_chinchilla_ablate_denom = fit_eqn(popt8, str_chinchilla_ablate_denom)
print("Fitted eqn1: ", str_fitted_eqn1)
print("Fitted eqn2: ", str_fitted_eqn2)
print("Fitted chinchilla: ", str_fitted_chinchilla)
print("Fitted modified chinchilla: ", str_fitted_modified_chinchilla)
print("Fitted chinchilla_ablation1: ", str_fitted_chinchilla_ablation1)
print("Fitted chinchilla_ablation2: ", str_fitted_chinchilla_ablation2)
print("Fitted chinchilla_ablate_numer: ", str_fitted_chinchilla_ablate_numer)
print("Fitted chinchilla_ablate_denom: ", str_fitted_chinchilla_ablate_denom)


fitted_eqn_1 = partial(eqn1, constant_1=popt1[0], constant_2=popt1[1])
fitted_eqn_2 = partial(eqn2, constant_1=popt2[0], constant_2=popt2[1], constant_3=popt2[2])
fitted_chinchilla = partial(chinchilla, constant_1=popt3[0], constant_2=popt3[1], constant_3=popt3[2], constant_4=popt3[3], constant_5=popt3[4])
fitted_modified_chinchilla = partial(modified_chinchilla, constant_1=popt4[0], constant_2=popt4[1], constant_3=popt4[2], constant_4=popt4[3], constant_5=popt4[4])
fitted_chinchilla_ablation1 = partial(chinchilla_ablation1, constant_1=popt5[0], constant_2=popt5[1], constant_3=popt5[2])
fitted_chinchilla_ablation2 = partial(chinchilla_ablation2, constant_1=popt6[0], constant_2=popt6[1], constant_3=popt6[2])
fitted_chinchilla_ablate_numer = partial(chinchilla_ablate_numer, constant_1=popt7[0], constant_2=popt7[1], constant_3=popt7[2])
fitted_chinchilla_ablate_denom = partial(chinchilla_ablate_denom, constant_1=popt8[0], constant_2=popt8[1], constant_3=popt8[2])

Fitted eqn1:  0.3658399805958976 + (-0.019393646403258875 / (training_steps ^ number_of_shots))
Fitted eqn2:  0.36586570526070605 + (-0.018970864489142484 / ((training_steps / 12604.999990734193) ^ number_of_shots))
Fitted chinchilla:  (1.0 / ((training_steps * training_batch_size) ^ 0.99983365)) + (0.8788157 / (total_params ^ 3.13948064)) + 0.3607812867216579
Fitted modified chinchilla:  (-0.019404684047361738 / ((training_steps * training_batch_size) ^ (1.00041618 * number_of_shots))) + (0.89328067 / (total_params ^ 2.88247603)) + 0.36584284907725884
Fitted chinchilla_ablation1:  (1.0 / ((training_steps * training_batch_size) ^ 1.0)) + 0.36078128665874637
Fitted chinchilla_ablation2:  (0.90374502 / (total_params ^ 2.6071086)) + 0.3607812868426495
Fitted chinchilla_ablate_numer:  (1 / ((training_steps * training_batch_size) ^ 0.99987481)) + (1 / (total_params ^ 2.60998853)) + 0.3607812868467931
Fitted chinchilla_ablate_denom:  (0.493647189 / (training_steps * training_batch_size)) + (

In [7]:
# get loss for each equation on the dataset

buckets = [0, 1e-7, 1e-5, 1e-1, 1, np.inf]

def get_loss(fitted_eqn, x, y):
    y_pred = fitted_eqn(x)
    diff = (y - y_pred) ** 2
    bucketed_loss = [0] * (len(buckets) - 1)
    for i in range(len(buckets) - 1):
        bucketed_loss[i] = len((diff[(diff > buckets[i]) & (diff <= buckets[i+1])]))
    return bucketed_loss, np.mean(diff), np.var(diff)


test_x = np.vstack([
    test_df['training_steps'].values,
    test_df['number_of_shots'].values,
])
test_wide_x = np.vstack([
    test_df['training_steps'].values,
    test_df['number_of_shots'].values,
    test_df['total_params'].values,
    test_df['training_batch_size'].values,
])

test_y = test_df['score'].values

loss1 = get_loss(fitted_eqn_1, test_x, test_y)
loss2 = get_loss(fitted_eqn_2, test_x, test_y)
loss3 = get_loss(fitted_chinchilla, test_wide_x, test_y)
loss4 = get_loss(fitted_modified_chinchilla, test_wide_x, test_y)
loss5 = get_loss(fitted_chinchilla_ablation1, test_wide_x, test_y)
loss6 = get_loss(fitted_chinchilla_ablation2, test_wide_x, test_y)
loss7 = get_loss(fitted_chinchilla_ablate_numer, test_wide_x, test_y)
loss8 = get_loss(fitted_chinchilla_ablate_denom, test_wide_x, test_y)
row_name = ['eqn1', 'eqn2', 'chinchilla', 'modified_chinchilla', 'chinchilla_ablation1', 'chinchilla_ablation2', 'chinchilla_ablate_numer', 'chinchilla_ablate_denom']
rows = [loss1, loss2, loss3, loss4, loss5, loss6, loss7, loss8]


header = ['<' + str(b) for b in buckets[1:]] + ['mean', 'variance']
print('{:<31}'.format('(y - y_pred)^2') + ' '.join([f'{k:<8}' for k in header]))
for i, (buckets, mean, variance) in enumerate(rows):
    print('{:<30}'.format(row_name[i]), ' '.join([f'{k:<8}' for k in buckets])+ f'{mean:<8.5f} {variance:<8.5f}')


(y - y_pred)^2                 <1e-07   <1e-05   <0.1     <1       <inf     mean     variance
eqn1                           4        39       9889     831      0       0.03643  0.00281 
eqn2                           5        41       9886     831      0       0.03644  0.00281 
chinchilla                     4        63       9931     765      0       0.03656  0.00284 
modified_chinchilla            4        39       9889     831      0       0.03643  0.00281 
chinchilla_ablation1           4        63       9931     765      0       0.03656  0.00284 
chinchilla_ablation2           4        63       9931     765      0       0.03656  0.00284 
chinchilla_ablate_numer        4        63       9931     765      0       0.03656  0.00284 
chinchilla_ablate_denom        3        47       9941     772      0       0.03643  0.00278 
